In [1]:
#import library
from libsbml import readSBML
from cobra import io
from cobra.core import Metabolite
from Functions import find_average

#import sbml file
model = io.sbml.create_cobra_model_from_sbml_file("/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_2_3.xml")

#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})

In [2]:
import pandas as pd
biomass = pd.DataFrame(data={"":["sSUCROSE_b","GLC_c","FRU_c","Starch_b","Cellulose_b","PALMITATE_c",
                                 "L_PHOSPHATIDATE_p","PHOSPHATIDYL_CHOLINE_r",
                                 "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","DIACYLGLYCEROL_p","Protein_b","sMAL_b",
                                 "sCIT_b","sFUM_b","ARG_c","HIS_c","LYS_c","sASP_b",
                                 "sGLU_b","sSER_b","THR_c","ASN_c","sGLN_b","CYS_c",
                                 "GLY_c","PRO_c","sALA_b","VAL_c","ILE_c","LEU_c",
                                 "MET_c","PHE_c","TYR_c","TRP_c","sGABA_b"],
                             "leaf":[0.0]*35,"stem":[0.0]*35,"root":[0.0]*35,"seed":[0.0]*35,},dtype="float64")
biomass = biomass.set_index("")

In [3]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

cobra/core/metabolite.py:103 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 25.9492553613 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 5.7497158698 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)


### Mature leaf

Extract CO2 assimilation rate for PPFD = 600 umol/m2/s between 11:00 and 13:00h from Liu et al 2004, Figure 3

<img src="references/Screenshot_20190717_132513.png" height=50% width=50%>

In [4]:
df_matureleaf_data = pd.DataFrame(data={"DAA":[-10.9638294010889,-9.92869328493648,
                                               -9.02599818511797,-7.9920054446461,
                                               -6.98697822141561,-4.05432849364791,
                                               -3.05117967332123,-2.04432849364791,
                                               -1.02035390199637,2.90665154264973,
                                               3.94565335753176,5.86627949183303,
                                               6.86583484573503,9.80232304900181],
                                        "Photosynthesis (umol/m2/s)":[18.4210526315789,18.2395644283121,
                                                                      14.6460980036298,15.989110707804,
                                                                      15.9528130671506,25.7531760435572,
                                                                      28.2214156079855,25.7531760435572,
                                                                      20.4537205081669,24.4464609800363,
                                                                      19.1107078039927,18.2758620689655,
                                                                      25.535390199637,30.2177858439201]})
df_matureleaf_data

DAA  Photosynthesis (umol/m2/s)
0  -10.963829                   18.421053
1   -9.928693                   18.239564
2   -9.025998                   14.646098
3   -7.992005                   15.989111
4   -6.986978                   15.952813
5   -4.054328                   25.753176
6   -3.051180                   28.221416
7   -2.044328                   25.753176
8   -1.020354                   20.453721
9    2.906652                   24.446461
10   3.945653                   19.110708
11   5.866279                   18.275862
12   6.865835                   25.535390
13   9.802323                   30.217786

Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [5]:
df_matureleaf_content_1 = pd.DataFrame(data={"Series":[1,2,3,4],
                                           "starch_dawn (g/100gFW)":[0.4245676464308603,0.4974291547005305,
                                                                     0.8895379007845832,1.2526663271935523],
                                           "hemicellulose_dawn (g/100gFW)":[1.089463383389083,
                                                                            1.8922736237500428,
                                                                            0.8616451473404596,
                                                                            1.1221330617548415],
                                           "simple sugars_dawn (g/100gFW)":[0.2719414988257076,
                                                                            0.6501251398452184,
                                                                            0.5686189976083096,
                                                                            0.5533809766290336],
                                           "sucrose_dawn (g/100gFW)":[0.2719414988257076,0.454127308166961,
                                                                      0.09770538968660913,0],
                                           "starch_dusk (g/100gFW)":[1.3284677247171013,1.600011533626287,
                                                                     2.6254282537133005,1.0978029358571406],
                                           "hemicellulose_dusk (g/100gFW)":[1.089463383389083,
                                                                            0.6633103814170216,
                                                                            0.9510687588805857,
                                                                            1.0721624730031074],
                                           "simple sugars_dusk (g/100gFW)":[0.03202263184115184,
                                                                            0.35337416237039254,
                                                                            0.4138784209550903,
                                                                            0.3891937806184267],
                                           "sucrose_dusk (g/100gFW)":[0.46007757454985465,0.18700390989930638,
                                                                      0.3964371762172485,0.14206905936682546]})
df_matureleaf_content_1.set_index("Series",drop=True,inplace=True)
df_matureleaf_content_1["Δstarch"] = df_matureleaf_content_1["starch_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["starch_dawn (g/100gFW)"]
df_matureleaf_content_1["Δhemicellulose"] = df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsimple sugars"] = df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"]
df_matureleaf_content_1["Δsucrose"] = df_matureleaf_content_1["sucrose_dusk (g/100gFW)"]- \
                                                        df_matureleaf_content_1["sucrose_dawn (g/100gFW)"]
df_matureleaf_content_1

hemicellulose_dawn (g/100gFW)  hemicellulose_dusk (g/100gFW)  \
Series                                                                 
1                            1.089463                       1.089463   
2                            1.892274                       0.663310   
3                            0.861645                       0.951069   
4                            1.122133                       1.072162   

        simple sugars_dawn (g/100gFW)  simple sugars_dusk (g/100gFW)  \
Series                                                                 
1                            0.271941                       0.032023   
2                            0.650125                       0.353374   
3                            0.568619                       0.413878   
4                            0.553381                       0.389194   

        starch_dawn (g/100gFW)  starch_dusk (g/100gFW)  \
Series                                                   
1                     0.424568                1.328468   
2                     0.497429                1.600012   
3                     0.889538                2.625428   
4                     1.252666                1.097803   

        sucrose_dawn (g/100gFW)  sucrose_dusk (g/100gFW)   Δstarch  \
Series                                                               
1                      0.271941                 0.460078  0.903900   
2                      0.454127                 0.187004  1.102582   
3                      0.097705                 0.396437  1.735890   
4                      0.000000                 0.142069 -0.154863   

        Δhemicellulose  Δsimple sugars  Δsucrose  
Series                                            
1             0.000000       -0.239919  0.188136  
2            -1.228963       -0.296751 -0.267123  
3             0.089424       -0.154741  0.298732  
4            -0.049971       -0.164187  0.142069

Series 3 ignored because of diurnal fluctuations in metabolite levels were not observed, calculate average from remaining data

In [6]:
df_matureleaf_content_1_std = pd.DataFrame(data={"avg. Δstarch (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δstarch"])],
                                                 "avg. Δstarch (%)":
                                                 [100*find_average(df_matureleaf_content_1["Δstarch"]/ \
                                                               df_matureleaf_content_1["starch_dusk (g/100gFW)"])],
                                                 "avg. Δsimple sugars (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δsimple sugars"])],
                                                 "avg. Δsucrose (g/100gFW)":
                                                 [find_average(df_matureleaf_content_1["Δsucrose"])],
                                                 "avg. Δsucrose (%)":
                                                  [100*find_average(df_matureleaf_content_1["Δsucrose"]/ \
                                                                df_matureleaf_content_1["sucrose_dusk (g/100gFW)"])]})
df_matureleaf_content_1_std

avg. Δsimple sugars (g/100gFW)  avg. Δstarch (%)  avg. Δstarch (g/100gFW)  \
0                       -0.213899         47.240852                 0.896877   

   avg. Δsucrose (%)  avg. Δsucrose (g/100gFW)  
0          18.350658                  0.090453

In [7]:
print("Starch to sucrose ratio = "+str(float(df_matureleaf_content_1_std["avg. Δstarch (g/100gFW)"]/ \
                                      df_matureleaf_content_1_std["avg. Δsucrose (g/100gFW)"])))

Starch to sucrose ratio = 9.9153546986


### Leaf

Extract sucrose,hexose and starch content from Liu et al 2004, Figure 4 (PPFD = 600 umol/m2/s, photoperiod=12h, sampled between 11:00 and 13:00 h into photoperiod)

<img src="references/Screenshot_20190717_135552.png" height=50% width=50%>

In [8]:
df_leaf_content_1 = pd.DataFrame(data={"DAA":[0,3,5,7,10],
                                       "sucrose (mg/gDW)":[6.3492063492063275,5.0793650793650515,
                                                           5.0793650793650515,4.20634920634918,
                                                           1.746031746031723],
                                       "hexose (mg/gDW)":[61.64383561643821,81.36986301369862,
                                                          82.46575342465738,69.3150684931507,
                                                          45.5707762557078],
                                       "starch (mg/gDW)":[47.42268041237085,47.01030927835024,
                                                          37.525773195875956,35.25773195876279,
                                                          64.32989690721634]})
df_leaf_content_1.set_index("DAA",inplace=True,drop=True)
df_leaf_content_1

hexose (mg/gDW)  starch (mg/gDW)  sucrose (mg/gDW)
DAA                                                    
0          61.643836        47.422680          6.349206
3          81.369863        47.010309          5.079365
5          82.465753        37.525773          5.079365
7          69.315068        35.257732          4.206349
10         45.570776        64.329897          1.746032

Using Clements 1930 data on starch accumulated in 12h to estimate starch, hexose and sucrose content when diel fluctuations are ignored.

In [9]:
df_leaf_content_1_revised = pd.DataFrame(data={"sucrose (mg/gDW)":df_leaf_content_1["sucrose (mg/gDW)"]* \
                                                  (100-df_matureleaf_content_1_std["avg. Δsucrose (%)"][0])/100,
                                           "starch (mg/gDW)":df_leaf_content_1["starch (mg/gDW)"]* \
                                                  (100-df_matureleaf_content_1_std["avg. Δstarch (%)"][0])/100,
                                           "hexose (mg/gDW)":df_leaf_content_1["hexose (mg/gDW)"]})
df_leaf_content_1_revised

hexose (mg/gDW)  starch (mg/gDW)  sucrose (mg/gDW)
DAA                                                    
0          61.643836        25.019802          5.184085
3          81.369863        24.802239          4.147268
5          82.465753        19.798278          4.147268
7          69.315068        18.601679          3.434456
10         45.570776        33.939906          1.425623

Gather stach, hexose and sucrose content, 8hrs after illumination from Huber 2008, Table 1

In [10]:
df_leaf_content_2 = pd.DataFrame(data={"cultivar":["Maple Presto","Ransom"],
                                       "starch (mg/dm2)":[190,78],
                                       "hexose (mg/dm2)":[0,1.9],
                                       "sucrose (mg/dm2)":[8.7,4.9]})
df_leaf_content_2.set_index("cultivar",drop=True,inplace=True)
df_leaf_content_2

hexose (mg/dm2)  starch (mg/dm2)  sucrose (mg/dm2)
cultivar                                                        
Maple Presto              0.0              190               8.7
Ransom                    1.9               78               4.9

Extract amino acid content from Chiozza et al 2010, Figure 2

<img src="references/Screenshot_20190717_143619.png" height=50% width=50%>

In [11]:
df_leaf_content_3 = pd.DataFrame(data={"Variety":["aphid resitant","aphid sensitive"],
                                       "GABA (nmol/50mgFW)":[0.8287671232876719,1.0136986301369868],
                                       "Asn (nmol/50mgFW)":[67.26027397260277,86.1643835616437],
                                       "Gln (nmol/50mgFW)":[82.87671232876716,99.31506849315056],
                                       "Glt (nmol/50mgFW)":[184.58904109589022,234.93150684931462],
                                       "His (nmol/50mgFW)":[11.778523489932848,13.657718120805297],
                                       "Pro (nmol/50mgFW)":[19.062499999999943,20.68181818181808],
                                       "Ser (nmol/50mgFW)":[87.28205128205093,100.4102564102559]})
df_leaf_content_3.set_index("Variety",drop=True,inplace=True)
df_leaf_content_3

Asn (nmol/50mgFW)  GABA (nmol/50mgFW)  Gln (nmol/50mgFW)  \
Variety                                                                     
aphid resitant           67.260274            0.828767          82.876712   
aphid sensitive          86.164384            1.013699          99.315068   

                 Glt (nmol/50mgFW)  His (nmol/50mgFW)  Pro (nmol/50mgFW)  \
Variety                                                                    
aphid resitant          184.589041          11.778523          19.062500   
aphid sensitive         234.931507          13.657718          20.681818   

                 Ser (nmol/50mgFW)  
Variety                             
aphid resitant           87.282051  
aphid sensitive         100.410256

Use Clements 1930 data to extract baseline sucrose, simple sugars, starch and hemicellulose data from diel metabolite levels

In [12]:
xlist = list()
ylist = list()
ylist2 = list()
ylist3 = list()
ylist4 = list()
for i in df_matureleaf_content_1.index:
    if i == 4:
        continue
    xlist.append(i)
    ylist.append(min(df_matureleaf_content_1["simple sugars_dawn (g/100gFW)"][i],
                     df_matureleaf_content_1["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_matureleaf_content_1["sucrose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["sucrose_dusk (g/100gFW)"][i]))
    ylist3.append(min(df_matureleaf_content_1["hemicellulose_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["hemicellulose_dusk (g/100gFW)"][i]))
    ylist4.append(min(df_matureleaf_content_1["starch_dawn (g/100gFW)"][i],
                      df_matureleaf_content_1["starch_dusk (g/100gFW)"][i]))
df_leaf_content_4 = pd.DataFrame(data={"Series":xlist,
                                       "baseline starch (g/100gFW)":ylist4,
                                       "baseline hemicellulose (g/100gFW)":ylist3,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_leaf_content_4.set_index("Series",drop=True,inplace=True)
df_leaf_content_4

baseline hemicellulose (g/100gFW)  baseline simple sugars (g/100gFW)  \
Series                                                                         
1                                1.089463                           0.032023   
2                                0.663310                           0.353374   
3                                0.861645                           0.413878   

        baseline starch (g/100gFW)  baseline sucrose (g/100gFW)  
Series                                                           
1                         0.424568                     0.271941  
2                         0.497429                     0.187004  
3                         0.889538                     0.097705

### Summary

- Since amino acid content was only available in g/gFW and FW/DW data was not available, diel starch and sucrose levels (in g/gFW) were used to predict baseline starch and sucrose levels in leaf


In [13]:
df_leaf_content_3

Asn (nmol/50mgFW)  GABA (nmol/50mgFW)  Gln (nmol/50mgFW)  \
Variety                                                                     
aphid resitant           67.260274            0.828767          82.876712   
aphid sensitive          86.164384            1.013699          99.315068   

                 Glt (nmol/50mgFW)  His (nmol/50mgFW)  Pro (nmol/50mgFW)  \
Variety                                                                    
aphid resitant          184.589041          11.778523          19.062500   
aphid sensitive         234.931507          13.657718          20.681818   

                 Ser (nmol/50mgFW)  
Variety                             
aphid resitant           87.282051  
aphid sensitive         100.410256

In [14]:
biomass.at["sSUCROSE_b","leaf"]=10*find_average(df_leaf_content_4["baseline sucrose (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["Starch_b","leaf"]=10*find_average(df_leaf_content_4["baseline starch (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","leaf"]=10*find_average(df_leaf_content_4["baseline simple sugars (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight

biomass.at["ASN_c","leaf"]= find_average(df_leaf_content_3["Asn (nmol/50mgFW)"])/50
biomass.at["sGABA_b","leaf"]= find_average(df_leaf_content_3["GABA (nmol/50mgFW)"])/50
biomass.at["sGLN_b","leaf"]= find_average(df_leaf_content_3["Gln (nmol/50mgFW)"])/50
biomass.at["sGLU_b","leaf"]= find_average(df_leaf_content_3["Glt (nmol/50mgFW)"])/50
biomass.at["HIS_c","leaf"]= find_average(df_leaf_content_3["His (nmol/50mgFW)"])/50
biomass.at["PRO_c","leaf"]= find_average(df_leaf_content_3["Pro (nmol/50mgFW)"])/50
biomass.at["sSER_b","leaf"]= find_average(df_leaf_content_3["Ser (nmol/50mgFW)"])/50

### Stem



Extract diurnal starch, sucrose, hemicellulose and simple sugar content from Clements 1930, Figure 6, 9, 13 and 14. Based on Figure 1 and 2, dawn = 8am and dusk = 8pm

<img src="references/Screenshot_20190717_161155.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160304.png" height=50% width=50%>

<img src="references/Screenshot_20190717_161259.png" height=50% width=50%>

<img src="references/Screenshot_20190717_160242.png" height=50% width=50%>

In [24]:
df_stem_content = pd.DataFrame(data={"Series":[1,2,3,4],
                                     "simple sugars_dawn (g/100gFW)":[0.08928305478569998,1.0069230769230781,
                                                                      0.22941176470588265,0.755800478743633],
                                     "sucrose_dawn (g/100gFW)":[0.24999412110523256,0.10918376068376556,
                                                                0.14117647058823524,0.36680114242250283],
                                     "simple sugars_dusk (g/100gFW)":[0.27423868312757205,0.9930341880341897,
                                                                      0.23368616214394122,0.8292719666520014],
                                     "sucrose_dusk (g/100gFW)":[0,0.08606410256411046,
                                                                0.1748626327321765,0.2553567833200341]})
df_stem_content.set_index("Series",drop=True,inplace=True)
df_stem_content["avg. Δsimple sugars (g/100gFW)"] = df_stem_content["simple sugars_dusk (g/100gFW)"]- \
                                                        df_stem_content["simple sugars_dawn (g/100gFW)"]
df_stem_content["avg. Δsucrose (g/100gFW)"] = df_stem_content["sucrose_dusk (g/100gFW)"]- \
                                                        df_stem_content["sucrose_dawn (g/100gFW)"]

xlist = list()
ylist = list()
ylist2 = list()
for i in df_matureleaf_content_1.index:
    xlist.append(i)
    ylist.append(min(df_stem_content["simple sugars_dawn (g/100gFW)"][i],
                     df_stem_content["simple sugars_dusk (g/100gFW)"][i]))
    ylist2.append(min(df_stem_content["sucrose_dawn (g/100gFW)"][i],
                      df_stem_content["sucrose_dusk (g/100gFW)"][i]))
df_stem_content_std = pd.DataFrame(data={"Series":xlist,
                                       "baseline sucrose (g/100gFW)":ylist2,
                                       "baseline simple sugars (g/100gFW)":ylist})
df_stem_content_std.set_index("Series",drop=True,inplace=True)
df_stem_content_std

baseline simple sugars (g/100gFW)  baseline sucrose (g/100gFW)
Series                                                                
1                                0.089283                     0.000000
2                                0.993034                     0.086064
3                                0.229412                     0.141176
4                                0.755800                     0.255357

### Summary

- Sucrose, hexose content data is per FW and conversion to DW is not available

Assumptions

In [25]:
biomass.at["sSUCROSE_b","stem"] = find_average(df_stem_content_std["baseline sucrose (g/100gFW)"])/ \
                                        model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","stem"]=find_average(df_stem_content_std["baseline simple sugars (g/100gFW)"])/ \
                                        model.metabolites.get_by_id("GLC_c").formula_weight

### Root

Gather root exudate content from Canarini et al 2016, Figure 2

<img src="references/Screenshot_20190718_145459.png" height=50% width=50%>

In [17]:
print("amount of C exudate ="+str(8.689956331877875)+" ug/gDWroot/2h")
print("amount of N exudate ="+str(1.315789473684226)+" ug/gDWroot/2h")

df_root_exudate_1 = pd.DataFrame(data={"%C or %N":["%C","%N"],"Sugars":[14.947368421052634,0],
                                       "Amino acids":[25.684210526315788 - 14.947368421052634,
                                                      67.89473684210535 - 25.789473684210833],
                                       "Organic acids":[30.105263157894733 - 25.684210526315788,0],
                                       "Inorganic ions":[0,67.89473684210535 - 25.789473684210833]})
df_root_exudate_1.set_index("%C or %N",inplace=True,drop=True)
df_root_exudate_1

amount of C exudate =8.68995633188 ug/gDWroot/2h
amount of N exudate =1.31578947368 ug/gDWroot/2h


Amino acids  Inorganic ions  Organic acids     Sugars
%C or %N                                                       
%C          10.736842        0.000000       4.421053  14.947368
%N          42.105263       42.105263       0.000000   0.000000

Gathering fatty-acid content from Thomas et al 2007 Table I

In [18]:
df_root_content_1 = pd.DataFrame(data={"Tissue type":["Conrad - Epidermis","OX760-6 - Epidermis",
                                                      "Conrad - Endodermis","OX760-6 - Endodermis"],
                                       "Fatty acid 16:1":[0.21,0.05,0.82,0.39],
                                       "Fatty acid 16:0":[0.39,0.25,1.50,0.72],
                                       "Fatty acid 18:1":[0.71,0.18,7.46,1.46],
                                       "Fatty acid 18:0":[0.32,0.04,0.83,0.30],
                                       "Fatty acid 20:0":[0.05,0.03,0.28,0.49],
                                       "Fatty acid 22:0":[0.09,0.04,0.31,0.31],
                                       "Total (ug/cm2)":[1.77,0.59,11.20,3.67]})
df_root_content_1.set_index("Tissue type",inplace=True,drop=True)
df_root_content_1

Fatty acid 16:0  Fatty acid 16:1  Fatty acid 18:0  \
Tissue type                                                               
Conrad - Epidermis               0.39             0.21             0.32   
OX760-6 - Epidermis              0.25             0.05             0.04   
Conrad - Endodermis              1.50             0.82             0.83   
OX760-6 - Endodermis             0.72             0.39             0.30   

                      Fatty acid 18:1  Fatty acid 20:0  Fatty acid 22:0  \
Tissue type                                                               
Conrad - Epidermis               0.71             0.05             0.09   
OX760-6 - Epidermis              0.18             0.03             0.04   
Conrad - Endodermis              7.46             0.28             0.31   
OX760-6 - Endodermis             1.46             0.49             0.31   

                      Total (ug/cm2)  
Tissue type                           
Conrad - Epidermis              1.77  
OX760-6 - Epidermis             0.59  
Conrad - Endodermis            11.20  
OX760-6 - Endodermis            3.67

### Summary

- sugars, amino acids and organic acids as %C and %N data could not be used directly.
- lipid composition in ug/cm2 cannot be translated to mg/gDW

### Seed


Gather non-reducing sugars, reducing sugars, starch, proteins, phospholipids, glycolipids, sterols and free fatty acid content from Guleria et al 2008, Table 1

In [19]:
df_seed_content = pd.DataFrame(data={"seed type":["large","small"],
                               "starch (mg/gDW)":[49.7,42.3],
                               "reducing sugars (mg/gDW)":[24.9,27.7],
                               "total soluble sugars (mg/gDW)":[125,101],
                               "proteins (mg/gDW)":[377,264],
                               "lipid content (mg/gDW)":[241,224],
                               "phospholipids (mg/gDW lipid)":[57.1,58.7],
                               "glycolipids (mg/gDW lipid)":[10.6,10.3],
                               "sterols (mg/gDW lipid)":[26.8,26.4],
                               "free fatty acids (mg/gDW lipid)":[0.1,0.1],
                               "16:0 (mg/gDW fatty acds)":[132,122],
                               "18:0 (mg/gDW fatty acds)":[29,38],
                               "18:1 (mg/gDW fatty acds)":[269,245],
                               "18:2 (mg/gDW fatty acds)":[513,518],
                               "18:3 (mg/gDW fatty acds)":[56,76]})
df_seed_content_std = pd.DataFrame(data={"seed type":["large","small"],
                               "starch (mg/gDW)":[49.7,42.3],
                               "reducing sugars (mg/gDW)":[24.9,27.7],
                               "non-reducing sugars (mg/gDW)":df_seed_content["total soluble sugars (mg/gDW)"]-\
                                         df_seed_content["reducing sugars (mg/gDW)"],
                               "proteins (mg/gDW)":[377,264],
                               "phospholipids (mg/gDW)":df_seed_content["phospholipids (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "glycolipids (mg/gDW)":df_seed_content["glycolipids (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "sterols (mg/gDW)":df_seed_content["sterols (mg/gDW lipid)"]* \
                                   df_seed_content["lipid content (mg/gDW)"],
                               "16:0 (mg/gDW)":df_seed_content["16:0 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:0 (mg/gDW)":df_seed_content["18:0 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:1 (mg/gDW)":df_seed_content["18:1 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:2 (mg/gDW)":df_seed_content["18:2 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"],
                               "18:3 (mg/gDW)":df_seed_content["18:3 (mg/gDW fatty acds)"]* \
                                         df_seed_content["free fatty acids (mg/gDW lipid)"]* \
                                         df_seed_content["lipid content (mg/gDW)"]})
df_seed_content_std.set_index("seed type",inplace=True,drop=True)
df_seed_content_std

16:0 (mg/gDW)  18:0 (mg/gDW)  18:1 (mg/gDW)  18:2 (mg/gDW)  \
seed type                                                               
large             3181.2          698.9         6482.9        12363.3   
small             2732.8          851.2         5488.0        11603.2   

           18:3 (mg/gDW)  glycolipids (mg/gDW)  non-reducing sugars (mg/gDW)  \
seed type                                                                      
large             1349.6                2554.6                         100.1   
small             1702.4                2307.2                          73.3   

           phospholipids (mg/gDW)  proteins (mg/gDW)  \
seed type                                              
large                     13761.1                377   
small                     13148.8                264   

           reducing sugars (mg/gDW)  starch (mg/gDW)  sterols (mg/gDW)  
seed type                                                               
large                          24.9             49.7            6458.8  
small                          27.7             42.3            5913.6

Extract FW/DW, lipid, protein, starch, soluble sugar and reducing sugar content from Sharma et al 2013, Figure 1 and 2


<img src="references/Screenshot_20190719_101524.png" height=25% width=25%>

<img src="references/Screenshot_20190719_101729.png" height=25% width=25%>

In [23]:
df_seed_content_2 = pd.DataFrame(data={"node position":range(4,18),
                               "DW/FW":[0.928872804935928,0.927787138111058,0.934610821072615,
                                        0.938872804935928,0.937322021831989,0.918329378262933,
                                        0.95515068818225,0.944299952539155,0.945538680588514,
                                        0.948406502135738,0.948249881347888,0.948559563360228,
                                        0.956778595158994,0.958483626008543],
                               "lipid content (mg/gDW)":[242.2236872277,264.320110066498,255.514790185737,
                                                         247.835932125659,237.345218986471,237.531529465719,
                                                         235.470648933731,225.550332492547,212.250630589314,
                                                         211.313345562944,207.564205457464,207.750515936712,
                                                         216.922724145838,221.603416647558],
                               "proteins (mg/gDW)":[251.291824869483,241.120372567632,245.770052206929,
                                                    222.503856193641,236.746559088752,239.656502135738,
                                                    269.587980541054,271.621381110584,278.882890365449,
                                                    334.986058376839,327.426435690555,323.350735643094,
                                                    316.663205980066,321.312885619364],
                               "starch (mg/gDW)":[42.5131154156578,40.2199354317999,48.283393866021,
                                                  37.240213882163,41.1970338983051,45.3324253430186,
                                                  42.681598062954,40.3894269572236,39.7033898305085,
                                                  49.195419693301,40.4736682808717,42.8233454398709,
                                                  39.8153753026634,35.5579096045198],
                               "total soluble sugars (mg/gDW)":[90.6887971663593,88.8516144140979,
                                                                88.4657943268639,105.697140013466,
                                                                81.2716255378942,91.6635929861538,
                                                                98.7394982582477,91.7227247445918,
                                                                86.7779046280847,86.8071777758262,
                                                                97.1985597611311,90.5968794824508,
                                                                81.5069816457364,90.6548403149791],
                               "reducing sugars (mg/gDW)":[112.999279159929,151.966724463752,
                                                           122.654639677375,137.206647314383,
                                                           155.484034366537,157.207036957665,
                                                           115.897056245007,65.4822809717704,
                                                           49.4141713261512,53.6188119776346,
                                                           58.6545617487192,78.173352295973,
                                                           46.7922616844281,52.2406436907015]})
df_seed_content_2.set_index("node position",drop=True,inplace=True)
df_seed_content_2

DW/FW  lipid content (mg/gDW)  proteins (mg/gDW)  \
node position                                                        
4              0.928873              242.223687         251.291825   
5              0.927787              264.320110         241.120373   
6              0.934611              255.514790         245.770052   
7              0.938873              247.835932         222.503856   
8              0.937322              237.345219         236.746559   
9              0.918329              237.531529         239.656502   
10             0.955151              235.470649         269.587981   
11             0.944300              225.550332         271.621381   
12             0.945539              212.250631         278.882890   
13             0.948407              211.313346         334.986058   
14             0.948250              207.564205         327.426436   
15             0.948560              207.750516         323.350736   
16             0.956779              216.922724         316.663206   
17             0.958484              221.603417         321.312886   

               reducing sugars (mg/gDW)  starch (mg/gDW)  \
node position                                              
4                            112.999279        42.513115   
5                            151.966724        40.219935   
6                            122.654640        48.283394   
7                            137.206647        37.240214   
8                            155.484034        41.197034   
9                            157.207037        45.332425   
10                           115.897056        42.681598   
11                            65.482281        40.389427   
12                            49.414171        39.703390   
13                            53.618812        49.195420   
14                            58.654562        40.473668   
15                            78.173352        42.823345   
16                            46.792262        39.815375   
17                            52.240644        35.557910   

               total soluble sugars (mg/gDW)  
node position                                 
4                                  90.688797  
5                                  88.851614  
6                                  88.465794  
7                                 105.697140  
8                                  81.271626  
9                                  91.663593  
10                                 98.739498  
11                                 91.722725  
12                                 86.777905  
13                                 86.807178  
14                                 97.198560  
15                                 90.596879  
16                                 81.506982  
17                                 90.654840

### Summary

Assumptions

- Sharma et al 2013 data has reducing  sugar content higher than total soluble sugar content and so given that the same data was available from Guleria et al 2008, this data was ignored

In [20]:
biomass.at["sSUCROSE_b","seed"] = find_average(df_seed_content_std["non-reducing sugars (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","seed"]=find_average(df_seed_content_std["reducing sugars (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["PALMITATE_c","seed"]=find_average(df_seed_content_std["16:0 (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("PALMITATE_c").formula_weight
biomass.at["L_PHOSPHATIDATE_p","seed"]=find_average(df_seed_content_std["phospholipids (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("L_PHOSPHATIDATE_p").formula_weight
biomass.at["Starch_b","seed"]=find_average(df_seed_content_std["starch (mg/gDW)"])/ \
                                        model.metabolites.get_by_id("GLC_c").formula_weight
biomass.at["Protein_b","seed"]=find_average(df_seed_content_std["proteins (mg/gDW)"])/proteinMW
biomass.to_csv("/home/sanu/biomass_soy.csv")
biomass

leaf  root       seed  stem
                                                                
sSUCROSE_b                       0.005421   0.0   0.253289   0.0
GLC_c                            0.007783   0.0   0.145985   0.0
FRU_c                            0.000000   0.0   0.000000   0.0
Starch_b                         0.017641   0.0   0.255334   0.0
Cellulose_b                      0.000000   0.0   0.000000   0.0
PALMITATE_c                      0.000000   0.0  11.577185   0.0
L_PHOSPHATIDATE_p                0.000000   0.0  20.799909   0.0
PHOSPHATIDYL_CHOLINE_r           0.000000   0.0   0.000000   0.0
L_1_PHOSPHATIDYL_ETHANOLAMINE_r  0.000000   0.0   0.000000   0.0
DIACYLGLYCEROL_p                 0.000000   0.0   0.000000   0.0
Protein_b                        0.000000   0.0   0.802620   0.0
sMAL_b                           0.000000   0.0   0.000000   0.0
sCIT_b                           0.000000   0.0   0.000000   0.0
sFUM_b                           0.000000   0.0   0.000000   0.0
ARG_c                            0.000000   0.0   0.000000   0.0
HIS_c                            0.254362   0.0   0.000000   0.0
LYS_c                            0.000000   0.0   0.000000   0.0
sASP_b                           0.000000   0.0   0.000000   0.0
sGLU_b                           4.195205   0.0   0.000000   0.0
sSER_b                           1.876923   0.0   0.000000   0.0
THR_c                            0.000000   0.0   0.000000   0.0
ASN_c                            1.534247   0.0   0.000000   0.0
sGLN_b                           1.821918   0.0   0.000000   0.0
CYS_c                            0.000000   0.0   0.000000   0.0
GLY_c                            0.000000   0.0   0.000000   0.0
PRO_c                            0.397443   0.0   0.000000   0.0
sALA_b                           0.000000   0.0   0.000000   0.0
VAL_c                            0.000000   0.0   0.000000   0.0
ILE_c                            0.000000   0.0   0.000000   0.0
LEU_c                            0.000000   0.0   0.000000   0.0
MET_c                            0.000000   0.0   0.000000   0.0
PHE_c                            0.000000   0.0   0.000000   0.0
TYR_c                            0.000000   0.0   0.000000   0.0
TRP_c                            0.000000   0.0   0.000000   0.0
sGABA_b                          0.018425   0.0   0.000000   0.0

### References

- Liu, F., Jensen, C.R. and Andersen, M.N. (2004) Drought stress effect on carbohydrate concentration in soybean leaves and pods during early reproductive development: its implication in altering pod set. F. Crop. Res., 86, 1–13. https://doi.org/10.1016/S0378-4290(03)00165-5.
- Chiozza, M. V, O’Neal, M.E. and MacIntosh, G.C. (2010) Constitutive and Induced Differential Accumulation of Amino Acid in Leaves of Susceptible and Resistant Soybean Plants in Response to the Soybean Aphid (Hemiptera: Aphididae). Environ. Entomol., 39, 856–864. https://doi.org/10.1603/EN09338.
- Huber, S.C. (2008) Biochemical Mechanism for Regulation of Sucrose Accumulation in Leaves during Photosynthesis. PLANT Physiol., 91, 656–662. https://doi.org/10.1104/pp.91.2.656.
- White, M.C., Decker, A.M. and Chaney, R.L. (2008) Metal Complexation in Xylem Fluid : I. CHEMICAL COMPOSITION OF TOMATO AND SOYBEAN STEM EXUDATE. PLANT Physiol., 67, 292–300. https://doi.org/10.1104/pp.67.2.292.
- Clements, H.F. (1930) Hourly Variations in Carbohydrate Content of Leaves and Petioles. Bot. Gaz., 89, 241–272. https://www.jstor.org/stable/2471041.
- Canarini, A., Merchant, A. and Dijkstra, F.A. (2016) Drought effects on Helianthus annuus and Glycine max metabolites: from phloem to root exudates. Rhizosphere, 2, 85–97. https://doi.org/10.1016/j.rhisph.2016.06.003.
- Thomas, R., Fang, X., Ranathunge, K., Anderson, T.R., Peterson, C.A. and Bernards, M.A. (2007) Soybean Root Suberin: Anatomical Distribution, Chemical Composition, and Relationship to Partial Resistance to Phytophthora sojae. PLANT Physiol., 144, 299–311. https:doi.org/10.1104/pp.106.091090.
- Guleria, S., Sharma, S., Gill, B.S. and Munshi, S.K. (2008) Distribution and biochemical composition of large and small seeds of soybean (Glycine max L.). J. Sci. Food Agric., 88, 269–272. https://doi.org/10.1002/jsfa.2825.
- Sharma, S., Kaur, A., Bansal, A. and Gill, B.S. (2013) Positional effects on soybean seed composition during storage. J. Food Sci. Technol., 50, 353–359. https://doi.org/10.1007/s13197-011-0341-0.